In [1]:
from __future__ import print_function
import IPython
import sys
from music21 import *
import numpy as np
from grammar import *
from qa import *
from preprocess import * 
from music_utils import *
from data_utils import *
from keras.models import load_model, Model
from keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector
from keras.initializers import glorot_uniform
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras import backend as K

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


AssertionError: 

In [2]:
from data_utils import *

StreamException: the object (<music21.stream.iterator.StreamIterator for Part:0x7fe44d7b4ba8 @:7>, id()=140618528772168) is already found in this Stream (<music21.stream.Part 0x7fe44d6ab780>, id()=140618528110464)

In [7]:
midi_data = converter.parse('data/original_metheny.mid')

In [10]:
melody_stream = midi_data[1] 

In [11]:
print(len(melody_stream))

1276


In [18]:
melody1 = melody_stream.getElementsByClass(stream.Voice)

In [26]:
def __parse_midi(data_fn):
    # Parse the MIDI data for separate melody and accompaniment parts.
    midi_data = converter.parse(data_fn)
    # Get melody part, compress into single voice.
    melody_stream = midi_data[1]     # For Metheny piece, Melody is Part #5.
    try:
     melody1, melody2 = melody_stream.getElementsByClass(stream.Voice)
     for j in melody2:
        melody1.insert(j.offset, j)
    except:   
      melody1 = melody_stream.getElementsByClass(stream.Voice)
    melody_voice = melody1

    for i in melody_voice:
        if i.quarterLength == 0.0:
            i.quarterLength = 0.25

    # Change key signature to adhere to comp_stream (1 sharp, mode = major).
    # Also add Electric Guitar. 
    melody_voice.insert(0, instrument.ElectricGuitar())
    melody_voice.insert(0, key.KeySignature(sharps=1))

    # The accompaniment parts. Take only the best subset of parts from
    # the original data. Maybe add more parts, hand-add valid instruments.
    # Should add least add a string part (for sparse solos).
    # Verified are good parts: 0, 1, 6, 7 '''
    partIndices = [0, 1, 6, 7]
    comp_stream = stream.Voice()
    comp_stream.append([j.flat for i, j in enumerate(midi_data) 
        if i in partIndices])

    # Full stream containing both the melody and the accompaniment. 
    # All parts are flattened. 
    full_stream = stream.Voice()
    for i in range(len(comp_stream)):
        full_stream.append(comp_stream[i])
    full_stream.append(melody_voice)

    # Extract solo stream, assuming you know the positions ..ByOffset(i, j).
    # Note that for different instruments (with stream.flat), you NEED to use
    # stream.Part(), not stream.Voice().
    # Accompanied solo is in range [478, 548)
    solo_stream = stream.Voice()
    for part in full_stream:
        curr_part = stream.Part()
        curr_part.append(part.getElementsByClass(instrument.Instrument))
        curr_part.append(part.getElementsByClass(tempo.MetronomeMark))
        curr_part.append(part.getElementsByClass(key.KeySignature))
        curr_part.append(part.getElementsByClass(meter.TimeSignature))
        curr_part.append(part.getElementsByOffset(476, 548, 
                                                  includeEndBoundary=True))
        cp = curr_part.flat
        solo_stream.insert(cp)

    # Group by measure so you can classify. 
    # Note that measure 0 is for the time signature, metronome, etc. which have
    # an offset of 0.0.
    melody_stream = solo_stream[-1]
    measures = OrderedDict()
    offsetTuples = [(int(n.offset / 4), n) for n in melody_stream]
    measureNum = 0 # for now, don't use real m. nums (119, 120)
    for key_x, group in groupby(offsetTuples, lambda x: x[0]):
        measures[measureNum] = [n[1] for n in group]
        measureNum += 1

    # Get the stream of chords.
    # offsetTuples_chords: group chords by measure number.
    chordStream = solo_stream[0]
    chordStream.removeByClass(note.Rest)
    chordStream.removeByClass(note.Note)
    offsetTuples_chords = [(int(n.offset / 4), n) for n in chordStream]

    # Generate the chord structure. Use just track 1 (piano) since it is
    # the only instrument that has chords. 
    # Group into 4s, just like before. 
    chords = OrderedDict()
    measureNum = 0
    for key_x, group in groupby(offsetTuples_chords, lambda x: x[0]):
        chords[measureNum] = [n[1] for n in group]
        measureNum += 1

    # Fix for the below problem.
    #   1) Find out why len(measures) != len(chords).
    #   ANSWER: resolves at end but melody ends 1/16 before last measure so doesn't
    #           actually show up, while the accompaniment's beat 1 right after does.
    #           Actually on second thought: melody/comp start on Ab, and resolve to
    #           the same key (Ab) so could actually just cut out last measure to loop.
    #           Decided: just cut out the last measure. 
    #del chords[len(chords) - 1]
    #assert len(chords) == len(measures)

    return measures, chords


In [24]:
#from preprocess import * 
from data_utils import *

In [ ]:
IPython.display.Audio('./data/30s_seq.mp3')

In [57]:
play_music('original_metheny.mid')

NameError: name 'play_music' is not defined

In [14]:
from music_utils import * 
from preprocess import * 
from keras.utils import to_categorical

chords, abstract_grammars = get_musical_data('data/original_metheny.mid')
corpus, tones, tones_indices, indices_tones = get_corpus_data(abstract_grammars)
N_tones = len(set(corpus))
n_a = 64
x_initializer = np.zeros((1, 1, 78))
a_initializer = np.zeros((1, n_a))
c_initializer = np.zeros((1, n_a))

def load_music_utils():
    chords, abstract_grammars = get_musical_data('data/original_metheny.mid')
    corpus, tones, tones_indices, indices_tones = get_corpus_data(abstract_grammars)
    N_tones = len(set(corpus))
    X, Y, N_tones = data_processing(corpus, tones_indices, 60, 30)   
    return (X, Y, N_tones, indices_tones)



ValueError: too many values to unpack (expected 2)

In [2]:
X, Y, n_values, indices_values = load_music_utils()
print(n_values)
print('shape of X:', X.shape)
print('shape of Y:', Y.shape)
print('number of training examples:', X.shape[0])
print('Tx (length of sequence):', X.shape[1])
print('total # of unique values:', n_values)
print('Shape of Y:', Y.shape)

78
shape of X: (60, 30, 78)
shape of Y: (30, 60, 78)
number of training examples: 60
Tx (length of sequence): 30
total # of unique values: 78
Shape of Y: (30, 60, 78)


In [7]:
n_a = 64 
reshapor = Reshape((1, 78))                        # Used in Step 2.B of djmodel(), below
#reshapor = Reshape((1, 1)) 
LSTM_cell = LSTM(n_a, return_state = True)         # Used in Step 2.C
densor = Dense(n_values, activation='softmax')     # Used in Step 2.D
#densor = Dense(58, activation='softmax') 

In [8]:
# GRADED FUNCTION: djmodel

def djmodel(Tx, n_a, n_values):
    """
    Implement the model
    
    Arguments:
    Tx -- length of the sequence in a corpus
    n_a -- the number of activations used in our model
    n_values -- number of unique values in the music data 
    
    Returns:
    model -- a keras model with the 
    """
    
    # Define the input of your model with a shape 
    X = Input(shape=(Tx, n_values))#(60,78)
    
    # Define s0, initial hidden state for the decoder LSTM
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    
    ### START CODE HERE ### 
    # Step 1: Create empty list to append the outputs while you iterate (≈1 line)
    outputs = []
    
    # Step 2: Loop
    for t in range(Tx):
        
        # Step 2.A: select the "t"th time step vector from X. 
        x = Lambda(lambda x: X[:,t,:])(X)
        # Step 2.B: Use reshapor to reshape x to be (1, n_values) (≈1 line)
        x = reshapor(x)
        # Step 2.C: Perform one step of the LSTM_cell
        a, _, c = LSTM_cell(x, initial_state=[a, c])
        # Step 2.D: Apply densor to the hidden state output of LSTM_Cell
        out = densor(a)
        # Step 2.E: add the output to "outputs"
        outputs.append(out)
        
    # Step 3: Create model instance
    model = Model(inputs=[X, a0, c0], outputs=outputs)
    
    ### END CODE HERE ###
    
    return model

In [9]:
model = djmodel(Tx = 30 , n_a = 64, n_values = 78)
#model = djmodel(Tx = 100 , n_a = 64, n_values = 1)

In [10]:
opt = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.01)

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [11]:
m = 60
#m = 58
a0 = np.zeros((m, n_a))
c0 = np.zeros((m, n_a))

In [16]:
print(len(X))

849


In [12]:
model.fit([X, a0, c0], list(Y), epochs=100)

Epoch 1/100
60/60 [==============================] - 12s 204ms/step - loss: 125.7586 - dense_1_loss_1: 4.3549 - dense_1_loss_2: 4.3508 - dense_1_loss_3: 4.3406 - dense_1_loss_4: 4.3399 - dense_1_loss_5: 4.3369 - dense_1_loss_6: 4.3377 - dense_1_loss_7: 4.3424 - dense_1_loss_8: 4.3351 - dense_1_loss_9: 4.3410 - dense_1_loss_10: 4.3378 - dense_1_loss_11: 4.3343 - dense_1_loss_12: 4.3293 - dense_1_loss_13: 4.3270 - dense_1_loss_14: 4.3273 - dense_1_loss_15: 4.3383 - dense_1_loss_16: 4.3294 - dense_1_loss_17: 4.3323 - dense_1_loss_18: 4.3375 - dense_1_loss_19: 4.3369 - dense_1_loss_20: 4.3387 - dense_1_loss_21: 4.3382 - dense_1_loss_22: 4.3282 - dense_1_loss_23: 4.3371 - dense_1_loss_24: 4.3350 - dense_1_loss_25: 4.3368 - dense_1_loss_26: 4.3273 - dense_1_loss_27: 4.3322 - dense_1_loss_28: 4.3375 - dense_1_loss_29: 4.3382 - dense_1_loss_30: 0.0000e+00 - dense_1_acc_1: 0.0000e+00 - dense_1_acc_2: 0.0333 - dense_1_acc_3: 0.1000 - dense_1_acc_4: 0.0333 - dense_1_acc_5: 0.0667 - dense_1_acc_6:

60/60 [==============================] - 0s 2ms/step - loss: 87.0138 - dense_1_loss_1: 4.1899 - dense_1_loss_2: 3.9190 - dense_1_loss_3: 3.6098 - dense_1_loss_4: 3.5404 - dense_1_loss_5: 3.2557 - dense_1_loss_6: 3.3309 - dense_1_loss_7: 3.2007 - dense_1_loss_8: 2.9048 - dense_1_loss_9: 2.9868 - dense_1_loss_10: 2.7740 - dense_1_loss_11: 2.9300 - dense_1_loss_12: 2.9986 - dense_1_loss_13: 2.7589 - dense_1_loss_14: 2.6624 - dense_1_loss_15: 2.8888 - dense_1_loss_16: 2.8381 - dense_1_loss_17: 2.7315 - dense_1_loss_18: 2.8468 - dense_1_loss_19: 2.6551 - dense_1_loss_20: 2.8800 - dense_1_loss_21: 2.8184 - dense_1_loss_22: 2.6309 - dense_1_loss_23: 2.7665 - dense_1_loss_24: 2.7153 - dense_1_loss_25: 3.0795 - dense_1_loss_26: 2.5446 - dense_1_loss_27: 2.7985 - dense_1_loss_28: 2.8073 - dense_1_loss_29: 2.9508 - dense_1_loss_30: 0.0000e+00 - dense_1_acc_1: 0.1000 - dense_1_acc_2: 0.1500 - dense_1_acc_3: 0.2000 - dense_1_acc_4: 0.1833 - dense_1_acc_5: 0.2667 - dense_1_acc_6: 0.1333 - dense_1_ac

60/60 [==============================] - 0s 2ms/step - loss: 53.8173 - dense_1_loss_1: 4.1002 - dense_1_loss_2: 3.4999 - dense_1_loss_3: 2.8035 - dense_1_loss_4: 2.4795 - dense_1_loss_5: 2.1034 - dense_1_loss_6: 2.0339 - dense_1_loss_7: 1.8905 - dense_1_loss_8: 1.8429 - dense_1_loss_9: 1.6912 - dense_1_loss_10: 1.5652 - dense_1_loss_11: 1.7177 - dense_1_loss_12: 1.5802 - dense_1_loss_13: 1.4225 - dense_1_loss_14: 1.5051 - dense_1_loss_15: 1.6730 - dense_1_loss_16: 1.6389 - dense_1_loss_17: 1.5452 - dense_1_loss_18: 1.5446 - dense_1_loss_19: 1.4998 - dense_1_loss_20: 1.5588 - dense_1_loss_21: 1.5548 - dense_1_loss_22: 1.4799 - dense_1_loss_23: 1.5791 - dense_1_loss_24: 1.5335 - dense_1_loss_25: 1.6667 - dense_1_loss_26: 1.5065 - dense_1_loss_27: 1.6666 - dense_1_loss_28: 1.5370 - dense_1_loss_29: 1.5972 - dense_1_loss_30: 0.0000e+00 - dense_1_acc_1: 0.0333 - dense_1_acc_2: 0.1833 - dense_1_acc_3: 0.3500 - dense_1_acc_4: 0.3000 - dense_1_acc_5: 0.3833 - dense_1_acc_6: 0.3667 - dense_1_ac

60/60 [==============================] - 0s 2ms/step - loss: 30.1081 - dense_1_loss_1: 4.0149 - dense_1_loss_2: 2.9840 - dense_1_loss_3: 1.9582 - dense_1_loss_4: 1.5450 - dense_1_loss_5: 1.2149 - dense_1_loss_6: 1.0520 - dense_1_loss_7: 0.9033 - dense_1_loss_8: 0.8973 - dense_1_loss_9: 0.8511 - dense_1_loss_10: 0.7516 - dense_1_loss_11: 0.8116 - dense_1_loss_12: 0.7309 - dense_1_loss_13: 0.6632 - dense_1_loss_14: 0.6800 - dense_1_loss_15: 0.7248 - dense_1_loss_16: 0.7515 - dense_1_loss_17: 0.6965 - dense_1_loss_18: 0.6913 - dense_1_loss_19: 0.6984 - dense_1_loss_20: 0.7819 - dense_1_loss_21: 0.7478 - dense_1_loss_22: 0.6781 - dense_1_loss_23: 0.7224 - dense_1_loss_24: 0.7329 - dense_1_loss_25: 0.7617 - dense_1_loss_26: 0.7130 - dense_1_loss_27: 0.7582 - dense_1_loss_28: 0.7780 - dense_1_loss_29: 0.8135 - dense_1_loss_30: 0.0000e+00 - dense_1_acc_1: 0.0667 - dense_1_acc_2: 0.3500 - dense_1_acc_3: 0.5500 - dense_1_acc_4: 0.6667 - dense_1_acc_5: 0.6500 - dense_1_acc_6: 0.8000 - dense_1_ac

60/60 [==============================] - 0s 2ms/step - loss: 16.7306 - dense_1_loss_1: 3.9524 - dense_1_loss_2: 2.4794 - dense_1_loss_3: 1.3034 - dense_1_loss_4: 0.7956 - dense_1_loss_5: 0.5665 - dense_1_loss_6: 0.4752 - dense_1_loss_7: 0.3947 - dense_1_loss_8: 0.3705 - dense_1_loss_9: 0.3294 - dense_1_loss_10: 0.3249 - dense_1_loss_11: 0.3256 - dense_1_loss_12: 0.2867 - dense_1_loss_13: 0.2681 - dense_1_loss_14: 0.2829 - dense_1_loss_15: 0.2943 - dense_1_loss_16: 0.3005 - dense_1_loss_17: 0.2860 - dense_1_loss_18: 0.2711 - dense_1_loss_19: 0.2778 - dense_1_loss_20: 0.3211 - dense_1_loss_21: 0.3129 - dense_1_loss_22: 0.2835 - dense_1_loss_23: 0.2870 - dense_1_loss_24: 0.2870 - dense_1_loss_25: 0.3210 - dense_1_loss_26: 0.2998 - dense_1_loss_27: 0.3189 - dense_1_loss_28: 0.3314 - dense_1_loss_29: 0.3828 - dense_1_loss_30: 0.0000e+00 - dense_1_acc_1: 0.0667 - dense_1_acc_2: 0.4667 - dense_1_acc_3: 0.7333 - dense_1_acc_4: 0.8500 - dense_1_acc_5: 0.9333 - dense_1_acc_6: 0.9667 - dense_1_ac

60/60 [==============================] - 0s 2ms/step - loss: 10.8830 - dense_1_loss_1: 3.9043 - dense_1_loss_2: 2.0641 - dense_1_loss_3: 0.8776 - dense_1_loss_4: 0.4073 - dense_1_loss_5: 0.2807 - dense_1_loss_6: 0.2201 - dense_1_loss_7: 0.1779 - dense_1_loss_8: 0.1649 - dense_1_loss_9: 0.1492 - dense_1_loss_10: 0.1292 - dense_1_loss_11: 0.1449 - dense_1_loss_12: 0.1231 - dense_1_loss_13: 0.1163 - dense_1_loss_14: 0.1135 - dense_1_loss_15: 0.1283 - dense_1_loss_16: 0.1357 - dense_1_loss_17: 0.1244 - dense_1_loss_18: 0.1169 - dense_1_loss_19: 0.1236 - dense_1_loss_20: 0.1364 - dense_1_loss_21: 0.1317 - dense_1_loss_22: 0.1231 - dense_1_loss_23: 0.1243 - dense_1_loss_24: 0.1218 - dense_1_loss_25: 0.1371 - dense_1_loss_26: 0.1361 - dense_1_loss_27: 0.1403 - dense_1_loss_28: 0.1467 - dense_1_loss_29: 0.1835 - dense_1_loss_30: 0.0000e+00 - dense_1_acc_1: 0.0667 - dense_1_acc_2: 0.5167 - dense_1_acc_3: 0.8167 - dense_1_acc_4: 0.9500 - dense_1_acc_5: 0.9667 - dense_1_acc_6: 1.0000 - dense_1_ac

60/60 [==============================] - 0s 2ms/step - loss: 8.4818 - dense_1_loss_1: 3.8633 - dense_1_loss_2: 1.7596 - dense_1_loss_3: 0.6310 - dense_1_loss_4: 0.2363 - dense_1_loss_5: 0.1681 - dense_1_loss_6: 0.1260 - dense_1_loss_7: 0.0962 - dense_1_loss_8: 0.0923 - dense_1_loss_9: 0.0830 - dense_1_loss_10: 0.0717 - dense_1_loss_11: 0.0766 - dense_1_loss_12: 0.0678 - dense_1_loss_13: 0.0640 - dense_1_loss_14: 0.0627 - dense_1_loss_15: 0.0679 - dense_1_loss_16: 0.0730 - dense_1_loss_17: 0.0679 - dense_1_loss_18: 0.0644 - dense_1_loss_19: 0.0650 - dense_1_loss_20: 0.0722 - dense_1_loss_21: 0.0714 - dense_1_loss_22: 0.0667 - dense_1_loss_23: 0.0660 - dense_1_loss_24: 0.0659 - dense_1_loss_25: 0.0735 - dense_1_loss_26: 0.0725 - dense_1_loss_27: 0.0768 - dense_1_loss_28: 0.0845 - dense_1_loss_29: 0.0955 - dense_1_loss_30: 0.0000e+00 - dense_1_acc_1: 0.0667 - dense_1_acc_2: 0.5500 - dense_1_acc_3: 0.8500 - dense_1_acc_4: 0.9833 - dense_1_acc_5: 0.9833 - dense_1_acc_6: 1.0000 - dense_1_acc

60/60 [==============================] - 0s 2ms/step - loss: 7.3232 - dense_1_loss_1: 3.8270 - dense_1_loss_2: 1.5441 - dense_1_loss_3: 0.4886 - dense_1_loss_4: 0.1627 - dense_1_loss_5: 0.1101 - dense_1_loss_6: 0.0838 - dense_1_loss_7: 0.0629 - dense_1_loss_8: 0.0605 - dense_1_loss_9: 0.0544 - dense_1_loss_10: 0.0477 - dense_1_loss_11: 0.0501 - dense_1_loss_12: 0.0448 - dense_1_loss_13: 0.0415 - dense_1_loss_14: 0.0406 - dense_1_loss_15: 0.0446 - dense_1_loss_16: 0.0476 - dense_1_loss_17: 0.0442 - dense_1_loss_18: 0.0429 - dense_1_loss_19: 0.0421 - dense_1_loss_20: 0.0469 - dense_1_loss_21: 0.0463 - dense_1_loss_22: 0.0439 - dense_1_loss_23: 0.0429 - dense_1_loss_24: 0.0433 - dense_1_loss_25: 0.0481 - dense_1_loss_26: 0.0470 - dense_1_loss_27: 0.0502 - dense_1_loss_28: 0.0551 - dense_1_loss_29: 0.0591 - dense_1_loss_30: 0.0000e+00 - dense_1_acc_1: 0.0667 - dense_1_acc_2: 0.6167 - dense_1_acc_3: 0.8833 - dense_1_acc_4: 1.0000 - dense_1_acc_5: 0.9833 - dense_1_acc_6: 1.0000 - dense_1_acc

60/60 [==============================] - 0s 2ms/step - loss: 6.6445 - dense_1_loss_1: 3.7943 - dense_1_loss_2: 1.3845 - dense_1_loss_3: 0.3946 - dense_1_loss_4: 0.1239 - dense_1_loss_5: 0.0794 - dense_1_loss_6: 0.0615 - dense_1_loss_7: 0.0461 - dense_1_loss_8: 0.0443 - dense_1_loss_9: 0.0397 - dense_1_loss_10: 0.0352 - dense_1_loss_11: 0.0368 - dense_1_loss_12: 0.0329 - dense_1_loss_13: 0.0303 - dense_1_loss_14: 0.0296 - dense_1_loss_15: 0.0327 - dense_1_loss_16: 0.0348 - dense_1_loss_17: 0.0319 - dense_1_loss_18: 0.0314 - dense_1_loss_19: 0.0304 - dense_1_loss_20: 0.0342 - dense_1_loss_21: 0.0339 - dense_1_loss_22: 0.0320 - dense_1_loss_23: 0.0316 - dense_1_loss_24: 0.0316 - dense_1_loss_25: 0.0354 - dense_1_loss_26: 0.0343 - dense_1_loss_27: 0.0368 - dense_1_loss_28: 0.0386 - dense_1_loss_29: 0.0421 - dense_1_loss_30: 0.0000e+00 - dense_1_acc_1: 0.0667 - dense_1_acc_2: 0.6333 - dense_1_acc_3: 0.9167 - dense_1_acc_4: 1.0000 - dense_1_acc_5: 1.0000 - dense_1_acc_6: 1.0000 - dense_1_acc

60/60 [==============================] - 0s 2ms/step - loss: 6.1888 - dense_1_loss_1: 3.7639 - dense_1_loss_2: 1.2590 - dense_1_loss_3: 0.3282 - dense_1_loss_4: 0.0995 - dense_1_loss_5: 0.0618 - dense_1_loss_6: 0.0478 - dense_1_loss_7: 0.0363 - dense_1_loss_8: 0.0346 - dense_1_loss_9: 0.0311 - dense_1_loss_10: 0.0277 - dense_1_loss_11: 0.0287 - dense_1_loss_12: 0.0258 - dense_1_loss_13: 0.0235 - dense_1_loss_14: 0.0230 - dense_1_loss_15: 0.0256 - dense_1_loss_16: 0.0272 - dense_1_loss_17: 0.0247 - dense_1_loss_18: 0.0244 - dense_1_loss_19: 0.0237 - dense_1_loss_20: 0.0265 - dense_1_loss_21: 0.0263 - dense_1_loss_22: 0.0249 - dense_1_loss_23: 0.0247 - dense_1_loss_24: 0.0246 - dense_1_loss_25: 0.0275 - dense_1_loss_26: 0.0263 - dense_1_loss_27: 0.0286 - dense_1_loss_28: 0.0301 - dense_1_loss_29: 0.0326 - dense_1_loss_30: 0.0000e+00 - dense_1_acc_1: 0.1000 - dense_1_acc_2: 0.6333 - dense_1_acc_3: 0.9167 - dense_1_acc_4: 1.0000 - dense_1_acc_5: 1.0000 - dense_1_acc_6: 1.0000 - dense_1_acc

In [44]:
print(len(X))

849


In [44]:
# GRADED FUNCTION: music_inference_model

def music_inference_model(LSTM_cell, densor, n_values = 78, n_a = 64, Ty = 100):
    """
    Uses the trained "LSTM_cell" and "densor" from model() to generate a sequence of values.
    
    Arguments:
    LSTM_cell -- the trained "LSTM_cell" from model(), Keras layer object
    densor -- the trained "densor" from model(), Keras layer object
    n_values -- integer, umber of unique values
    n_a -- number of units in the LSTM_cell
    Ty -- integer, number of time steps to generate
    
    Returns:
    inference_model -- Keras model instance
    """
    
    # Define the input of your model with a shape 
    x0 = Input(shape=(1, n_values))
    
    # Define s0, initial hidden state for the decoder LSTM
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    x = x0

    ### START CODE HERE ###
    # Step 1: Create an empty list of "outputs" to later store your predicted values (≈1 line)
    outputs = []
    
    # Step 2: Loop over Ty and generate a value at every time step
    for t in range(Ty):
        
        # Step 2.A: Perform one step of LSTM_cell (≈1 line)
        a, _, c = LSTM_cell(x, initial_state=[a, c])
        
        # Step 2.B: Apply Dense layer to the hidden state output of the LSTM_cell (≈1 line)
        out = densor(a)

        # Step 2.C: Append the prediction "out" to "outputs". out.shape = (None, 78) (≈1 line)
        outputs.append(out)
        
        # Step 2.D: Select the next value according to "out", and set "x" to be the one-hot representation of the
        #           selected value, which will be passed as the input to LSTM_cell on the next step. We have provided 
        #           the line of code you need to do this. 
        x = Lambda(one_hot)(out)
        
    # Step 3: Create model instance with the correct "inputs" and "outputs" (≈1 line)
    inference_model = Model(inputs=[x0, a0, c0], outputs=outputs)
    
    ### END CODE HERE ###
    
    return inference_model

In [45]:
inference_model = music_inference_model(LSTM_cell, densor, n_values = 78, n_a = 64, Ty = 50)

In [46]:
x_initializer = np.zeros((1, 1, 78))
a_initializer = np.zeros((1, n_a))
c_initializer = np.zeros((1, n_a))

In [47]:
# GRADED FUNCTION: predict_and_sample

def predict_and_sample(inference_model, x_initializer = x_initializer, a_initializer = a_initializer, 
                       c_initializer = c_initializer):
    """
    Predicts the next value of values using the inference model.
    
    Arguments:
    inference_model -- Keras model instance for inference time
    x_initializer -- numpy array of shape (1, 1, 78), one-hot vector initializing the values generation
    a_initializer -- numpy array of shape (1, n_a), initializing the hidden state of the LSTM_cell
    c_initializer -- numpy array of shape (1, n_a), initializing the cell state of the LSTM_cel
    
    Returns:
    results -- numpy-array of shape (Ty, 78), matrix of one-hot vectors representing the values generated
    indices -- numpy-array of shape (Ty, 1), matrix of indices representing the values generated
    """
    
    ### START CODE HERE ###
    # Step 1: Use your inference model to predict an output sequence given x_initializer, a_initializer and c_initializer.
    pred = inference_model.predict([x_initializer, a_initializer, c_initializer])
    # Step 2: Convert "pred" into an np.array() of indices with the maximum probabilities
    indices = np.argmax(pred, axis=-1)
    # Step 3: Convert indices to one-hot vectors, the shape of the results should be (1, )
    results = to_categorical(indices, num_classes=78)
    ### END CODE HERE ###
    
    return results, indices

In [48]:
results, indices = predict_and_sample(inference_model, x_initializer, a_initializer, c_initializer)
print("np.argmax(results[12]) =", np.argmax(results[12]))
print("np.argmax(results[17]) =", np.argmax(results[17]))
print("list(indices[12:18]) =", list(indices[12:18]))

np.argmax(results[12]) = 59
np.argmax(results[17]) = 73
list(indices[12:18]) = [array([59]), array([67]), array([59]), array([3]), array([31]), array([73])]


In [50]:
out_stream = generate_music(inference_model)

Predicting new values for different set of chords.
Generated 51 sounds using the predicted values for the set of chords ("1") and after pruning
Generated 51 sounds using the predicted values for the set of chords ("2") and after pruning
Generated 51 sounds using the predicted values for the set of chords ("3") and after pruning
Generated 51 sounds using the predicted values for the set of chords ("4") and after pruning
Generated 51 sounds using the predicted values for the set of chords ("5") and after pruning
Your generated music is saved in output/my_music.midi


####Trail

In [46]:
import glob
from music21 import converter, instrument, note, chord

notes = []

for file in glob.glob("midi_songs/*.mid"):
    midi = converter.parse(file)
    notes_to_parse = None

    parts = instrument.partitionByInstrument(midi)

    if parts: # file has instrument parts
        notes_to_parse = parts.parts[0].recurse()
    else: # file has notes in a flat structure
        notes_to_parse = midi.flat.notes

    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

In [4]:
#print(notes)

In [47]:
import numpy
from keras.utils import np_utils
sequence_length = 100

# get all pitch names
pitchnames = sorted(set(item for item in notes))

# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

network_input = []
network_output = []

# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])

n_patterns = len(network_input)
n_vocab=n_patterns
# reshape the input into a format compatible with LSTM layers
network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
# normalize input
network_input = network_input #/ float(n_vocab)

network_output = np_utils.to_categorical(network_output)

In [48]:
#print(network_input)

In [49]:
#print(network_output)

In [32]:
X=network_input
Y=network_output

In [50]:
print(network_input.shape)
print(network_output.shape)

(3165, 100, 1)
(3165, 143)


In [51]:
def get_musical_data(data_fn):
    
    measures, chords = __parse_midi(data_fn)
    abstract_grammars = __get_abstract_grammars(measures, chords)

    return chords, abstract_grammars

''' Get corpus data from grammatical data '''
def get_corpus_data(abstract_grammars):
    corpus = [x for sublist in abstract_grammars for x in sublist.split(' ')]
    values = set(corpus)
    val_indices = dict((v, i) for i, v in enumerate(values))
    indices_val = dict((i, v) for i, v in enumerate(values))

    return corpus, values, val_indices, indices_val

In [4]:
chords, abstract_grammars = get_musical_data('data/original_metheny.mid')

In [22]:
#chords, abstract_grammars = get_musical_data('midi_songs/StereoLove.mid')
midi_data = converter.parse('data/Bhairav.mid')
for part in midi_data.parts:
    print(part.partName)

Sitar
Piano


In [2]:
midi_data = converter.parse('data/music2.mid')

NameError: name 'converter' is not defined

In [17]:
print(midi_data[0])

<music21.stream.Part 0x7f9ddc9a99b0>


In [67]:
print((midi_data[2]))

<music21.stream.Part 0x7f95b5c8c828>


In [4]:
print(abstract_grammars)

['C,0.500 C,0.333,<P1,d-5> C,0.250,<M-2,m-6> C,0.250,<M3,d-3> C,0.250,<d6,m2> C,0.250,<d5,P1> C,0.667,<M2,d-4> C,0.250,<d1,P-5> C,0.250,<P-4,d-8> S,0.250,<d4,M-2> S,0.250,<P5,A1> C,0.250,<d5,P1>', 'C,0.500 S,0.250,<m2,P-4> C,0.250,<P4,m-2> A,0.250,<P4,m-2> S,0.500,<d1,P-5> C,0.250,<P1,d-5> C,0.250,<m2,P-4> A,0.250,<m2,P-4> C,0.250,<M2,d-4> A,0.250,<d4,M-2> C,0.250,<P4,m-2> C,0.250,<P4,m-2>', 'C,0.250 C,0.250,<d2,A-4> A,0.250,<m-2,d-6> C,0.250,<d3,M-3> A,0.250,<M3,d-3> C,0.250,<d6,m2> C,0.250,<A4,d-2> C,0.250,<d3,M-3> C,0.250,<m2,P-4> A,0.250,<m2,P-4> S,0.250,<P1,d-5> C,0.250,<m2,P-4> C,0.250,<P1,d-5> C,0.250,<m2,P-4> A,0.250,<m2,P-4>', 'C,0.250 S,0.250,<d6,m2> X,0.250,<A4,d-2> S,0.250,<dd5,d1> C,0.250,<P4,m-2> C,0.250,<M3,d-3> C,0.250,<dd5,d1> S,0.250,<P4,m-2> C,0.250,<P4,m-2> C,0.250,<M3,d-3> C,0.250,<dd5,d1> A,0.250,<M3,d-3>', 'C,0.500 A,0.333,<M2,d-4> C,0.250,<d4,M-2> A,0.333,<M2,d-4> C,0.250,<m2,P-4> C,0.250,<A-4,P-8> C,0.250,<P4,m-2> A,0.250,<P4,m-2> S,0.250,<d5,P1> A,0.333,<P1,d-

In [8]:
print(chords)

OrderedDict([(0, [<music21.instrument.Piano Piano>, <music21.instrument.ElectricGuitar Electric Guitar>, <music21.tempo.MetronomeMark Quarter=112.0>, <music21.key.Key of G major>, <music21.key.KeySignature of 1 sharp>, <music21.meter.TimeSignature 4/4>]), (1, [<music21.chord.Chord E-4 G4 C4 B-3 G#2>, <music21.chord.Chord B-3 F4 D4 A3>]), (2, [<music21.chord.Chord E-4 G3 G#3 C4>, <music21.chord.Chord F4 B-3 D4 G3>]), (3, [<music21.chord.Chord E-4 G#3 C4 F3>, <music21.chord.Chord F4 D4 B-3 G3>]), (4, [<music21.chord.Chord G#3 C#4 F3>, <music21.chord.Chord E-4 C#4 C4>, <music21.chord.Chord B-3 E-3 C4 G3 G#1>]), (5, [<music21.chord.Chord C4 A3 E4 G3 F2>, <music21.chord.Chord E4 C4 A3 G3 F2>, <music21.chord.Chord F3 B-3 A3 D4>]), (6, [<music21.chord.Chord D2 C4 E3 F3 A3>, <music21.chord.Chord B-2 F4 C4 D4 A4>]), (7, [<music21.chord.Chord B-3 E-3 F4 D4 G3>, <music21.chord.Chord F3 A3 C4 E4 D2>, <music21.chord.Chord A3 C4 E4 F3>, <music21.chord.Chord F3 A3 E4 C4 D2>, <music21.chord.Chord B-3 

In [3]:
print(chords)

OrderedDict([(0, [<music21.instrument.Piano Piano>, <music21.tempo.MetronomeMark Quarter=112.0>, <music21.key.Key of G major>, <music21.meter.TimeSignature 4/4>]), (1, [<music21.chord.Chord E-4 G4 C4 B-3 G#2>, <music21.chord.Chord B-3 F4 D4 A3>]), (2, [<music21.chord.Chord E-4 G3 G#3 C4>, <music21.chord.Chord F4 B-3 D4 G3>]), (3, [<music21.chord.Chord E-4 G#3 C4 F3>, <music21.chord.Chord F4 D4 B-3 G3>]), (4, [<music21.chord.Chord G#3 C#4 F3>, <music21.chord.Chord E-4 C#4 C4>, <music21.chord.Chord B-3 E-3 C4 G3 G#1>]), (5, [<music21.chord.Chord C4 A3 E4 G3 F2>, <music21.chord.Chord E4 C4 A3 G3 F2>, <music21.chord.Chord F3 B-3 A3 D4>]), (6, [<music21.chord.Chord D2 C4 E3 F3 A3>, <music21.chord.Chord B-2 F4 C4 D4 A4>, <music21.chord.Chord B-3 E-3 F4 D4 G3>]), (7, [<music21.chord.Chord F3 A3 C4 E4 D2>, <music21.chord.Chord A3 C4 E4 F3>, <music21.chord.Chord F3 A3 E4 C4 D2>, <music21.chord.Chord B-3 F4 D4 A3>]), (8, [<music21.chord.Chord B-3 A3 F3 D4>]), (9, [<music21.chord.Chord C4 A3 E4 G

In [2]:
print(abstract_grammars)

[]
